# Notebook 113: 学習率スケジューリング

## Learning Rate Scheduling

---

### このノートブックの位置づけ

**Phase 10「最適化手法」** の第4章として、学習の進行に応じて学習率を調整する **スケジューリング** 手法を学びます。

### 学習目標

1. **Step Decay** の仕組みを理解する
2. **Exponential Decay** を実装する
3. **Cosine Annealing** を理解する
4. **Warmup** の重要性を理解する
5. **Cyclical Learning Rate** を学ぶ

### 前提知識

- Notebook 110-112 の内容

---

## 目次

1. [学習率スケジューリングの必要性](#1-学習率スケジューリングの必要性)
2. [Step Decay](#2-step-decay)
3. [Exponential Decay](#3-exponential-decay)
4. [Cosine Annealing](#4-cosine-annealing)
5. [Warmup](#5-warmup)
6. [Cyclical Learning Rate](#6-cyclical-learning-rate)
7. [One Cycle Policy](#7-one-cycle-policy)
8. [スケジューラの比較](#8-スケジューラの比較)
9. [演習問題](#9-演習問題)
10. [まとめと次のステップ](#10-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. 学習率スケジューリングの必要性

### 1.1 固定学習率の問題

- **学習初期**: 大きな学習率で素早く探索したい
- **学習後期**: 小さな学習率で精密に収束したい

固定学習率では両立が難しい。

In [ ]:
# 固定学習率の問題を可視化

def noisy_quadratic(x, noise_scale=0.5):
    """ノイズ付き二次関数"""
    return (x - 2)**2 + noise_scale * np.random.randn()

def noisy_quadratic_grad(x, noise_scale=0.5):
    """ノイズ付き勾配"""
    return 2 * (x - 2) + noise_scale * np.random.randn()


# 異なる固定学習率での学習
lrs = [0.01, 0.1, 0.5]
n_steps = 200

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, lr in zip(axes, lrs):
    np.random.seed(42)
    x = 0.0
    history = [x]
    
    for _ in range(n_steps):
        grad = noisy_quadratic_grad(x)
        x = x - lr * grad
        history.append(x)
    
    ax.plot(history)
    ax.axhline(y=2, color='red', linestyle='--', label='最適値')
    ax.set_xlabel('ステップ')
    ax.set_ylabel('x')
    ax.set_title(f'学習率 = {lr}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【観察】")
print("- 小さい学習率(0.01): 収束が遅い")
print("- 中間の学習率(0.1): バランスが良い")
print("- 大きい学習率(0.5): 収束後も振動")

---

## 2. Step Decay

### 2.1 アイデア

一定のエポック数ごとに学習率を減少させます。

$$
\eta_t = \eta_0 \cdot \gamma^{\lfloor t / S \rfloor}
$$

- $\eta_0$: 初期学習率
- $\gamma$: 減衰率（例: 0.1）
- $S$: ステップサイズ（例: 30 epochs）

In [ ]:
class StepLR:
    """
    Step Decay Learning Rate Scheduler
    
    η = η₀ * γ^(epoch // step_size)
    """
    
    def __init__(self, initial_lr, step_size, gamma=0.1):
        self.initial_lr = initial_lr
        self.step_size = step_size
        self.gamma = gamma
    
    def get_lr(self, epoch):
        return self.initial_lr * (self.gamma ** (epoch // self.step_size))


# Step Decayの可視化
scheduler = StepLR(initial_lr=0.1, step_size=30, gamma=0.1)
epochs = np.arange(100)
lrs = [scheduler.get_lr(e) for e in epochs]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(epochs, lrs, 'b-', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('学習率')
ax.set_title('Step Decay (step_size=30, γ=0.1)')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)

# ステップ位置をマーク
for s in [30, 60, 90]:
    ax.axvline(x=s, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("【Step Decay】")
print(f"Epoch 0-29:  lr = {scheduler.get_lr(0)}")
print(f"Epoch 30-59: lr = {scheduler.get_lr(30)}")
print(f"Epoch 60-89: lr = {scheduler.get_lr(60)}")

### 2.2 MultiStepLR

指定したエポックで減衰させるバリエーション。

In [ ]:
class MultiStepLR:
    """
    Multi-Step Learning Rate Scheduler
    """
    
    def __init__(self, initial_lr, milestones, gamma=0.1):
        self.initial_lr = initial_lr
        self.milestones = sorted(milestones)
        self.gamma = gamma
    
    def get_lr(self, epoch):
        lr = self.initial_lr
        for milestone in self.milestones:
            if epoch >= milestone:
                lr *= self.gamma
        return lr


# MultiStepLRの可視化
scheduler = MultiStepLR(initial_lr=0.1, milestones=[30, 60, 80], gamma=0.1)
epochs = np.arange(100)
lrs = [scheduler.get_lr(e) for e in epochs]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(epochs, lrs, 'b-', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('学習率')
ax.set_title('MultiStepLR (milestones=[30, 60, 80])')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)

for s in [30, 60, 80]:
    ax.axvline(x=s, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

---

## 3. Exponential Decay

### 3.1 アイデア

各エポックで学習率を指数的に減衰させます。

$$
\eta_t = \eta_0 \cdot \gamma^t
$$

In [ ]:
class ExponentialLR:
    """
    Exponential Decay Learning Rate Scheduler
    
    η = η₀ * γ^epoch
    """
    
    def __init__(self, initial_lr, gamma=0.95):
        self.initial_lr = initial_lr
        self.gamma = gamma
    
    def get_lr(self, epoch):
        return self.initial_lr * (self.gamma ** epoch)


# 可視化
gammas = [0.9, 0.95, 0.99]
epochs = np.arange(100)

fig, ax = plt.subplots(figsize=(10, 5))

for gamma in gammas:
    scheduler = ExponentialLR(initial_lr=0.1, gamma=gamma)
    lrs = [scheduler.get_lr(e) for e in epochs]
    ax.plot(epochs, lrs, linewidth=2, label=f'γ = {gamma}')

ax.set_xlabel('Epoch')
ax.set_ylabel('学習率')
ax.set_title('Exponential Decay')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 4. Cosine Annealing

### 4.1 アイデア

学習率をコサイン関数に従って滑らかに減衰させます。

$$
\eta_t = \eta_{\min} + \frac{1}{2}(\eta_{\max} - \eta_{\min})\left(1 + \cos\left(\frac{t}{T_{\max}}\pi\right)\right)
$$

In [ ]:
class CosineAnnealingLR:
    """
    Cosine Annealing Learning Rate Scheduler
    """
    
    def __init__(self, initial_lr, T_max, eta_min=0):
        self.initial_lr = initial_lr
        self.T_max = T_max
        self.eta_min = eta_min
    
    def get_lr(self, epoch):
        return self.eta_min + 0.5 * (self.initial_lr - self.eta_min) * \
               (1 + np.cos(np.pi * epoch / self.T_max))


# Cosine Annealingの可視化
scheduler = CosineAnnealingLR(initial_lr=0.1, T_max=100, eta_min=0.001)
epochs = np.arange(100)
lrs = [scheduler.get_lr(e) for e in epochs]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(epochs, lrs, 'b-', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('学習率')
ax.set_title('Cosine Annealing (T_max=100)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【Cosine Annealingの利点】")
print("- 滑らかな減衰により学習が安定")
print("- Step Decayのような急激な変化がない")
print("- 学習終盤で精密な収束が可能")

### 4.2 Cosine Annealing with Warm Restarts

周期的に学習率をリセットするバリエーション。局所解からの脱出に効果的。

In [ ]:
class CosineAnnealingWarmRestarts:
    """
    Cosine Annealing with Warm Restarts
    """
    
    def __init__(self, initial_lr, T_0, T_mult=1, eta_min=0):
        self.initial_lr = initial_lr
        self.T_0 = T_0
        self.T_mult = T_mult
        self.eta_min = eta_min
    
    def get_lr(self, epoch):
        # 現在のサイクルと、サイクル内での位置を計算
        if self.T_mult == 1:
            T_cur = epoch % self.T_0
            T_i = self.T_0
        else:
            # 幾何級数の場合
            n = 0
            T_sum = 0
            while T_sum + self.T_0 * (self.T_mult ** n) <= epoch:
                T_sum += self.T_0 * (self.T_mult ** n)
                n += 1
            T_i = self.T_0 * (self.T_mult ** n)
            T_cur = epoch - T_sum
        
        return self.eta_min + 0.5 * (self.initial_lr - self.eta_min) * \
               (1 + np.cos(np.pi * T_cur / T_i))


# Warm Restartsの可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# T_mult = 1 (固定周期)
scheduler1 = CosineAnnealingWarmRestarts(initial_lr=0.1, T_0=20, T_mult=1, eta_min=0.001)
epochs = np.arange(100)
lrs1 = [scheduler1.get_lr(e) for e in epochs]

axes[0].plot(epochs, lrs1, 'b-', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('学習率')
axes[0].set_title('Warm Restarts (T_0=20, T_mult=1)')
axes[0].grid(True, alpha=0.3)

# T_mult = 2 (周期が倍増)
scheduler2 = CosineAnnealingWarmRestarts(initial_lr=0.1, T_0=10, T_mult=2, eta_min=0.001)
lrs2 = [scheduler2.get_lr(e) for e in epochs]

axes[1].plot(epochs, lrs2, 'r-', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('学習率')
axes[1].set_title('Warm Restarts (T_0=10, T_mult=2)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 5. Warmup

### 5.1 アイデア

学習初期は小さな学習率から始め、徐々に目標の学習率まで上げていきます。

**なぜ必要か**:
- 初期の勾配は不安定（重みがランダム初期化されている）
- 大きな学習率で始めると発散するリスク
- 特に大規模モデル（Transformer等）で重要

In [ ]:
class LinearWarmup:
    """
    Linear Warmup Scheduler
    """
    
    def __init__(self, target_lr, warmup_epochs):
        self.target_lr = target_lr
        self.warmup_epochs = warmup_epochs
    
    def get_lr(self, epoch):
        if epoch < self.warmup_epochs:
            return self.target_lr * (epoch + 1) / self.warmup_epochs
        return self.target_lr


class WarmupCosineAnnealing:
    """
    Warmup + Cosine Annealing
    """
    
    def __init__(self, target_lr, warmup_epochs, total_epochs, eta_min=0):
        self.target_lr = target_lr
        self.warmup_epochs = warmup_epochs
        self.total_epochs = total_epochs
        self.eta_min = eta_min
    
    def get_lr(self, epoch):
        if epoch < self.warmup_epochs:
            # Linear warmup
            return self.target_lr * (epoch + 1) / self.warmup_epochs
        else:
            # Cosine annealing
            progress = (epoch - self.warmup_epochs) / (self.total_epochs - self.warmup_epochs)
            return self.eta_min + 0.5 * (self.target_lr - self.eta_min) * \
                   (1 + np.cos(np.pi * progress))


# Warmupの可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear Warmup only
scheduler1 = LinearWarmup(target_lr=0.1, warmup_epochs=10)
epochs = np.arange(100)
lrs1 = [scheduler1.get_lr(e) for e in epochs]

axes[0].plot(epochs, lrs1, 'b-', linewidth=2)
axes[0].axvline(x=10, color='red', linestyle='--', alpha=0.5, label='Warmup終了')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('学習率')
axes[0].set_title('Linear Warmup (warmup=10)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Warmup + Cosine
scheduler2 = WarmupCosineAnnealing(target_lr=0.1, warmup_epochs=10, total_epochs=100, eta_min=0.001)
lrs2 = [scheduler2.get_lr(e) for e in epochs]

axes[1].plot(epochs, lrs2, 'b-', linewidth=2)
axes[1].axvline(x=10, color='red', linestyle='--', alpha=0.5, label='Warmup終了')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('学習率')
axes[1].set_title('Warmup + Cosine Annealing')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【Warmupの効果】")
print("- 初期の不安定な勾配による発散を防ぐ")
print("- 大規模モデル（BERT, ViT等）では必須")
print("- 一般的に全体の5-10%程度のエポック")

---

## 6. Cyclical Learning Rate

### 6.1 アイデア

学習率を周期的に変動させることで、局所解からの脱出を促進します。

In [ ]:
class CyclicalLR:
    """
    Cyclical Learning Rate (triangular policy)
    """
    
    def __init__(self, base_lr, max_lr, step_size):
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
    
    def get_lr(self, epoch):
        cycle = np.floor(1 + epoch / (2 * self.step_size))
        x = np.abs(epoch / self.step_size - 2 * cycle + 1)
        return self.base_lr + (self.max_lr - self.base_lr) * max(0, 1 - x)


class CyclicalLR2:
    """
    Cyclical Learning Rate (triangular2 policy)
    最大学習率が各サイクルで半減
    """
    
    def __init__(self, base_lr, max_lr, step_size):
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
    
    def get_lr(self, epoch):
        cycle = np.floor(1 + epoch / (2 * self.step_size))
        x = np.abs(epoch / self.step_size - 2 * cycle + 1)
        # 各サイクルで最大値を半減
        amplitude = (self.max_lr - self.base_lr) / (2 ** (cycle - 1))
        return self.base_lr + amplitude * max(0, 1 - x)


# Cyclical LRの可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs = np.arange(100)

# Triangular
scheduler1 = CyclicalLR(base_lr=0.001, max_lr=0.1, step_size=20)
lrs1 = [scheduler1.get_lr(e) for e in epochs]

axes[0].plot(epochs, lrs1, 'b-', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('学習率')
axes[0].set_title('Cyclical LR (triangular)')
axes[0].grid(True, alpha=0.3)

# Triangular2
scheduler2 = CyclicalLR2(base_lr=0.001, max_lr=0.1, step_size=20)
lrs2 = [scheduler2.get_lr(e) for e in epochs]

axes[1].plot(epochs, lrs2, 'r-', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('学習率')
axes[1].set_title('Cyclical LR (triangular2)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 7. One Cycle Policy

### 7.1 アイデア

Leslie Smith提案の手法。学習全体を通じて1サイクルだけ学習率を変動させます。

1. 小さな学習率から始める
2. 最大学習率まで増加
3. 最小学習率まで減少

In [ ]:
class OneCycleLR:
    """
    One Cycle Learning Rate Policy
    """
    
    def __init__(self, max_lr, total_epochs, pct_start=0.3, div_factor=25, final_div_factor=1e4):
        self.max_lr = max_lr
        self.total_epochs = total_epochs
        self.pct_start = pct_start
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor
        
        self.initial_lr = max_lr / div_factor
        self.min_lr = max_lr / final_div_factor
        self.up_epochs = int(total_epochs * pct_start)
        self.down_epochs = total_epochs - self.up_epochs
    
    def get_lr(self, epoch):
        if epoch < self.up_epochs:
            # 上昇フェーズ
            progress = epoch / self.up_epochs
            return self.initial_lr + (self.max_lr - self.initial_lr) * progress
        else:
            # 下降フェーズ（コサイン減衰）
            progress = (epoch - self.up_epochs) / self.down_epochs
            return self.min_lr + 0.5 * (self.max_lr - self.min_lr) * \
                   (1 + np.cos(np.pi * progress))


# One Cycle Policyの可視化
scheduler = OneCycleLR(max_lr=0.1, total_epochs=100, pct_start=0.3)
epochs = np.arange(100)
lrs = [scheduler.get_lr(e) for e in epochs]

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(epochs, lrs, 'b-', linewidth=2)
ax.axvline(x=30, color='red', linestyle='--', alpha=0.5, label='ピーク (30%)')
ax.set_xlabel('Epoch')
ax.set_ylabel('学習率')
ax.set_title('One Cycle Policy (pct_start=0.3)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【One Cycle Policy】")
print(f"初期学習率: {scheduler.initial_lr:.6f}")
print(f"最大学習率: {scheduler.max_lr}")
print(f"最小学習率: {scheduler.min_lr:.8f}")
print("")
print("Super-Convergence: 従来より少ないエポックで同等以上の精度を達成")

---

## 8. スケジューラの比較

### 8.1 全スケジューラの比較

In [ ]:
# 全スケジューラの比較

schedulers = {
    'Step Decay': StepLR(0.1, 30, 0.1),
    'Exponential': ExponentialLR(0.1, 0.95),
    'Cosine': CosineAnnealingLR(0.1, 100, 0.001),
    'Warmup+Cosine': WarmupCosineAnnealing(0.1, 10, 100, 0.001),
    'Cyclical': CyclicalLR(0.001, 0.1, 20),
    'One Cycle': OneCycleLR(0.1, 100, 0.3),
}

epochs = np.arange(100)

fig, ax = plt.subplots(figsize=(12, 6))

for name, scheduler in schedulers.items():
    lrs = [scheduler.get_lr(e) for e in epochs]
    ax.plot(epochs, lrs, linewidth=2, label=name)

ax.set_xlabel('Epoch')
ax.set_ylabel('学習率')
ax.set_title('学習率スケジューラの比較')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 8.2 スケジューラ選択ガイド

In [ ]:
print("="*80)
print("学習率スケジューラ選択ガイド")
print("="*80)
print("")
print("【Step Decay】")
print("  用途: 画像分類（ResNet等の古典的手法）")
print("  特徴: シンプルで実績がある")
print("  典型: lr=0.1, milestones=[30,60,90], γ=0.1")
print("")
print("【Cosine Annealing】")
print("  用途: 汎用的、特にファインチューニング")
print("  特徴: 滑らかな減衰、チューニングが容易")
print("  典型: T_max=total_epochs, eta_min=0 or 1e-6")
print("")
print("【Warmup + Cosine】")
print("  用途: Transformer, ViT等の大規模モデル")
print("  特徴: 初期の不安定性を回避")
print("  典型: warmup=5-10%のエポック")
print("")
print("【One Cycle】")
print("  用途: 高速な学習（Super-Convergence）")
print("  特徴: 少ないエポックで高精度")
print("  典型: pct_start=0.3, div_factor=25")
print("")
print("【Cyclical LR】")
print("  用途: 局所解からの脱出が必要な場合")
print("  特徴: 最適な学習率範囲の発見にも使用")
print("  典型: LR Range Testで範囲を決定")

---

## 9. 演習問題

### 演習 9.1: Polynomial Decay の実装

多項式減衰スケジューラを実装してください。

$$
\eta_t = (\eta_0 - \eta_{\min}) \left(1 - \frac{t}{T}\right)^p + \eta_{\min}
$$

In [ ]:
# 演習 9.1: 解答欄

class PolynomialLR:
    def __init__(self, initial_lr, total_epochs, power=1.0, end_lr=0.0):
        # TODO: 実装
        pass
    
    def get_lr(self, epoch):
        # TODO: 実装
        pass

# TODO: p=1, p=2, p=0.5 を比較

### 演習 9.2: LR Range Test

学習率を指数的に増加させながら損失を記録し、最適な学習率範囲を見つけるテストを実装してください。

In [ ]:
# 演習 9.2: 解答欄

def lr_range_test(model_fn, grad_fn, start_lr=1e-7, end_lr=10, num_steps=100):
    """
    LR Range Test
    
    Args:
        model_fn: 損失関数
        grad_fn: 勾配関数
        start_lr: 開始学習率
        end_lr: 終了学習率
        num_steps: ステップ数
    
    Returns:
        lrs: 学習率のリスト
        losses: 損失のリスト
    """
    # TODO: 実装
    pass

# TODO: テスト

### 演習 9.3: カスタムスケジューラ

2つのスケジューラを組み合わせたカスタムスケジューラを実装してください。
例: 最初の50%はLinear Warmup、残りの50%はExponential Decay

In [ ]:
# 演習 9.3: 解答欄

class ChainedScheduler:
    # TODO: 実装
    pass

---

## 10. まとめと次のステップ

### このノートブックで学んだこと

1. **Step Decay**: 一定間隔で学習率を減衰

2. **Exponential Decay**: 毎エポック指数的に減衰

3. **Cosine Annealing**: 滑らかなコサイン減衰

4. **Warmup**: 学習初期に徐々に学習率を上げる

5. **Cyclical LR**: 周期的に変動させる

6. **One Cycle**: 1サイクルで上昇→下降

### 次のノートブック（114: 正則化と最適化）への橋渡し

学習率スケジューリングは過学習を防ぐ効果もありますが、より直接的な正則化手法があります：

- **L1/L2正則化**
- **Weight Decay**
- **Dropout**

次のノートブックでは、これらの正則化と最適化の関係を学びます。

---

## 参考文献

1. Smith, L. N. (2017). Cyclical learning rates for training neural networks. *WACV*.
2. Smith, L. N., & Topin, N. (2019). Super-convergence: Very fast training using large learning rates. *arXiv:1708.07120*.
3. Loshchilov, I., & Hutter, F. (2017). SGDR: Stochastic gradient descent with warm restarts. *ICLR*.
4. Gotmare, A., et al. (2019). A closer look at deep learning heuristics: Learning rate restarts, warmup and distillation. *ICLR*.